In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
from torch.utils.tensorboard import SummaryWriter
import numpy as np
import cv2
from PIL import Image

In [27]:
input_size = 28
num_classes = 10
num_epochs = 20
batch_size = 128

train_dataset = torchvision.datasets.MNIST(root='data',
                              train=True,
                              transform=torchvision.transforms.ToTensor())

test_dataset = torchvision.datasets.MNIST(root='data',
                             train=False,
                             transform=torchvision.transforms.ToTensor())

In [5]:
#构建 mini-batch
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)

In [22]:
#构建 网络结构
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Sequential(nn.Conv2d(in_channels=1,
                                             out_channels=8,
                                             kernel_size=5,
                                             stride=1,
                                             padding=2),
                                   nn.ReLU(),
                                   nn.MaxPool2d(kernel_size=2),)
        self.conv2 = nn.Sequential(nn.Conv2d(8, 16, 5, 1, 2),
                                   nn.ReLU(),
                                   nn.MaxPool2d(2),)
        self.out = nn.Linear(16 * 7 * 7, 10)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)
        output = self.out(x)
        return output

In [7]:
#定义准确率作为评估标准
def accuracy(predictions, labels):
    pred = torch.max(predictions.data, 1)[1]            #torch.max()[0]返回最大值 torch.max()[1]返回最大值位置，既预测值
    rights = pred.eq(labels.data.view_as(pred)).sum()
    return rights, len(labels)                          #这里返回的是正确个数与总个数组成的元组

In [29]:
net = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)
net.cuda()

for epoch in range(num_epochs):
    train_rights = []
    
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.cuda(), target.cuda()
        net.train()
        output = torch.softmax(net(data), dim=1)
        loss = criterion(output, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        right = accuracy(output, target)
        train_rights.append(right)
        
        if batch_idx % 100 == 0:
            net.eval()
            val_rights = []
            
            for (data, target) in test_loader:
                data, target = data.cuda(), target.cuda()#测试数据传入显存
                output = net(data)
                right = accuracy(output, target)
                val_rights.append(right)
            
            train_r = (sum([tup[0] for tup in train_rights]), sum([tup[1] for tup in train_rights]))
            val_r = (sum([tup[0] for tup in val_rights]), sum([tup[1] for tup in val_rights]))
            
            print(f'当前epoch：{epoch} [{batch_idx*batch_size}/{len(train_loader.dataset)}]   损失：{loss.data:.6f}  训练集准确率：{100*train_r[0]/train_r[1]:.2f}%   测试集准确率：{100*val_r[0]/val_r[1]:.2f}%')

当前epoch：0 [0/60000]   损失：2.302480  训练集准确率：12.50%   测试集准确率：11.97%
当前epoch：0 [12800/60000]   损失：1.707443  训练集准确率：60.14%   测试集准确率：81.31%
当前epoch：0 [25600/60000]   损失：1.598603  训练集准确率：71.16%   测试集准确率：84.05%
当前epoch：0 [38400/60000]   损失：1.589960  训练集准确率：75.41%   测试集准确率：84.87%
当前epoch：0 [51200/60000]   损失：1.607264  训练集准确率：77.67%   测试集准确率：85.95%
当前epoch：1 [0/60000]   损失：1.620426  训练集准确率：84.38%   测试集准确率：86.64%
当前epoch：1 [12800/60000]   损失：1.600156  训练集准确率：85.94%   测试集准确率：87.01%
当前epoch：1 [25600/60000]   损失：1.605415  训练集准确率：86.43%   测试集准确率：87.22%
当前epoch：1 [38400/60000]   损失：1.529931  训练集准确率：86.94%   测试集准确率：95.37%
当前epoch：1 [51200/60000]   损失：1.510738  训练集准确率：88.97%   测试集准确率：95.78%
当前epoch：2 [0/60000]   损失：1.529776  训练集准确率：93.75%   测试集准确率：96.04%
当前epoch：2 [12800/60000]   损失：1.497789  训练集准确率：95.92%   测试集准确率：96.83%
当前epoch：2 [25600/60000]   损失：1.492424  训练集准确率：96.29%   测试集准确率：96.91%
当前epoch：2 [38400/60000]   损失：1.491920  训练集准确率：96.31%   测试集准确率：97.06%
当前epoch：2 [51200/60000]   损失：1.491367  训练集准确率：

In [30]:
def plot_conv(writer, model):
    for name,param in model.named_parameters():
        if 'conv' in name and 'weight' in name:
            in_channels = param.size()[1]	# 输入通道
            out_channels = param.size()[0]   # 输出通道
            k_w, k_h = param.size()[3], param.size()[2]   # 卷积核的尺寸
            kernel_all = param.view(-1, 1, k_w, k_h)  # 每个通道的卷积核
            kernel_grid = torchvision.utils.make_grid(kernel_all, normalize=True, scale_each=True, nrow=in_channels)
            writer.add_image(f'{name}_all', kernel_grid, global_step=0)

In [31]:
writer = SummaryWriter("logs")
plot_conv(writer, net)